In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#    https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
### limitations under the License.

Use case:

In [13]:
# [START aiplatform_sdk_embedding]
!pip install google-cloud-aiplatform==1.35.0 --upgrade --quiet --user

In [15]:
import sys

if "google.colab" in sys.modules:
    # Define project information
    PROJECT_ID = "abis-345004"  # @param {type:"string"}
    LOCATION = "us-central1"  # @param {type:"string"}

    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()


In [16]:
 import vertexai
 vertexai.init(project=PROJECT_ID, location="us-central1")

In [17]:
from vertexai.language_models import TextEmbeddingModel

In [ ]:
!pip install gradio
import gradio as gr

In [18]:
def text_embedding(content) -> list:
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@latest")
    embeddings = model.get_embeddings(content)
    for embedding in embeddings:
        vector = embedding.values
        #print(f"Length of Embedding Vector: {len(vector)}")
    return vector

In [29]:
from google.cloud import aiplatform
DEPLOYED_INDEX_ID = "spanner_vector1_1702366982123"

In [30]:
def vector_search(content) -> list:
  result = text_embedding(content)
  #call_vector_search_api(content)
  index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/273845608377/locations/us-central1/indexEndpoints/2021628049526620160')
  # run query
  response = index_endpoint.find_neighbors(
      deployed_index_id = DEPLOYED_INDEX_ID,
      queries = [result],
      num_neighbors = 10
  )
  out = []
  # show the results
  for idx, neighbor in enumerate(response[0]):
      print(f"{neighbor.distance:.2f} {spanner_read_data(neighbor.id)}")
      out.append(f"{spanner_read_data(neighbor.id)}")
  return out

In [ ]:
#requirements.txt
!pip install google-cloud-spanner==3.36.0

In [40]:
from google.cloud import spanner

instance_id = "spanner-vertex"
database_id = "spanner-vertex-embeddings"
projectId = PROJECT_ID

client = spanner.Client()
client.project = projectId
instance = client.instance(instance_id)
database = instance.database(database_id)

def spanner_read_data(id):
    #query = "SELECT id, content, replace(replace(uri,'gs://img_public_test/data files/Apparel/girls-images_with_product_ids/','https://storage.googleapis.com/img_public_test/data%20files/Apparel/'),'gs://img_public_test/data files/Apparel/boys-images_with_product_ids/', 'https://storage.googleapis.com/img_public_test/data%20files/Apparel/') as uri FROM apparel_main where id = '" + id + "'"
    query = "SELECT uri FROM apparels where id = " + id
    outputs = []
    with database.snapshot() as snapshot:
        results = snapshot.execute_sql(query)

        for row in results:
            #print(row)
            #output = "ID: {}, CONTENT: {}, URI: {}".format(*row)
            output = "{}".format(*row)
            outputs.append(output)

    return "\n".join(outputs)


In [43]:
from PIL import Image

def call_search(query):
  response = vector_search(query)
  return response

#Query = gr.inputs.Textbox(label="Enter question")
#Response = gr.outputs.Textbox(label="Response")
input_text = gr.Textbox(label="Enter your query. Examples: Orange girls tops, denim skirt etc.")
output_texts = [gr.Image(label="") for i in range(10)]
demo = gr.Interface(fn=call_search, inputs=input_text, outputs=output_texts, live=True)
resp = demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://48b836d9ec1d688bfd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
